In [1]:
import os
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import plotly.express as px
%matplotlib inline


from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.feature_selection import mutual_info_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
# from cuml.preprocessing import StandardScaler


sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'

# import cudf # insted of pandas so it use gpu



from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [2]:
data= pd.read_csv('/kaggle/input/iiiiiotv2/cleaned_data.csv')
data

,L4_SRC_PORT,L4_DST_PORT,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,SERVER_TCP_FLAGS,SRC_TO_DST_SECOND_BYTES,DST_TO_SRC_SECOND_BYTES,RETRANSMITTED_IN_BYTES,...,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_OUT,DNS_QUERY_TYPE,DNS_TTL_ANSWER,IPV4_SRC_ADDR_INT,IIPV4_DST_ADDR_INT,Attack_Num
0,49235,4444,155392,202,34552,149,24,155392.0,34552.0,0,...,150,36,14,95,4805,0,0,3232235969,3232235809,9
1,49228,1880,1600,40,35741,65,24,1600.0,35741.0,0,...,3,30,19,6,237,0,0,3232235969,3232235928,0
2,0,0,212,2,0,0,0,212.0,0.0,0,...,0,0,0,0,0,0,0,3232235928,3232235969,0
3,65317,1900,165,1,0,0,0,165.0,0.0,0,...,1,0,0,0,0,0,0,3232235945,4026531834,0
4,60766,15600,63,1,0,0,0,63.0,0.0,0,...,0,0,0,0,0,0,0,3232235855,3232236031,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16940491,1880,49178,31067,49,1160,29,16,31067.0,1160.0,0,...,1,14,16,9,16425,0,0,3232235928,3232235969,0
16940492,43,37796,4275,5,296,5,17,4275.0,296.0,4171,...,0,0,0,3,1024,0,0,3088660875,3232235956,0
16940493,443,49198,191,4,151,3,25,191.0,151.0,0,...,0,0,0,0,63846,0,0,878257681,3232235969,0
16940494,53,57992,72,1,0,0,0,72.0,0.0,0,...,0,0,0,0,0,12,0,3232235966,3232235969,0


# regresion'

In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

def remove_outliers_iqr(df):
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        Q1 = df[col].quantile(0.05)
        Q3 = df[col].quantile(0.95)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.6 * IQR
        upper_bound = Q3 + 1.6 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

cleaned_data = remove_outliers_iqr(data)

scaler = StandardScaler()
scaled_data = scaler.fit_transform(cleaned_data)

scaled_df = pd.DataFrame(scaled_data, columns=cleaned_data.columns)

print(scaled_df.head())

   L4_SRC_PORT  L4_DST_PORT  IN_BYTES   IN_PKTS  OUT_BYTES  OUT_PKTS  \
0    -3.546541    -0.559583  0.258460  0.477859  -0.367578 -0.704575   
1    -3.546541    -0.559583  0.258460  0.477859  -0.367578 -0.704575   
2    -0.246231    -0.551255  0.413623  1.967549   0.034614  1.427878   
3    -0.392049    -0.551255  0.413623  1.967549   0.034614  1.427878   
4    -0.670329    -0.551255  0.413623  1.967549   0.034614  1.427878   

   SERVER_TCP_FLAGS  SRC_TO_DST_SECOND_BYTES  DST_TO_SRC_SECOND_BYTES  \
0         -0.883823                 0.242612                -0.366901   
1         -0.883823                 0.242612                -0.366901   
2          1.548139                 0.392742                 0.030077   
3          1.548139                 0.392742                 0.030077   
4          1.548139                 0.392742                 0.030077   

   RETRANSMITTED_IN_BYTES  ...  NUM_PKTS_128_TO_256_BYTES  \
0                     0.0  ...                  -0.239647   
1     

In [11]:
X = scaled_df.drop(columns=['Attack_Num'])
y = scaled_df['Attack_Num']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
print(X_train.shape, X_test.shape) 

In [7]:
 model = ExtraTreesRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_test_pred)
mse = mean_squared_error(y_test, y_test_pred)
rmse = mse ** 0.5  
r2 = r2_score(y_test, y_test_pred)

train_accuracy = model.score(X_train, y_train)
test_accuracy = model.score(X_test, y_test)

print(f"Training Accuracy (R² Score): {train_accuracy:.4f}")
print(f"Testing Accuracy (R² Score): {test_accuracy:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")

Training Accuracy (R² Score): 0.9950
Testing Accuracy (R² Score): 0.9796
Mean Absolute Error (MAE): 0.0178
Mean Squared Error (MSE): 0.0204
Root Mean Squared Error (RMSE): 0.1428
R² Score: 0.9796


✅ MAE কম হলে, মডেল সঠিকভাবে প্রেডিক্ট করছে।

✅ RMSE ও MSE কম হলে, মডেল কম error করছে।

✅ R² Score 1.0-এর কাছাকাছি হলে, মডেল ভালোভাবে data fit করছে।

# multiclass Classification

In [3]:
def remove_outliers_iqr(df):
    df_filtered = df.copy()
    for col in df_filtered.select_dtypes(include=['int64', 'float64']).columns:
        Q1 = df_filtered[col].quantile(0.05)
        Q3 = df_filtered[col].quantile(0.95)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.6 * IQR
        upper_bound = Q3 + 1.6 * IQR
        df_filtered = df_filtered.loc[(df_filtered[col] >= lower_bound) & (df_filtered[col] <= upper_bound)]
    
    return df_filtered

cleaned_data = remove_outliers_iqr(data)
if 'Attack_Num' not in cleaned_data.columns:
    raise ValueError("Target column 'Attack_Num' missing after outlier removal")

X = cleaned_data.drop(columns=['Attack_Num'])
y = cleaned_data['Attack_Num']  

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_scaled_df, y, test_size=0.1, random_state=42)

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)

X_train shape: (12036894, 23)
X_test shape: (1337433, 23)


In [4]:
X

,L4_SRC_PORT,L4_DST_PORT,IN_BYTES,IN_PKTS,OUT_BYTES,OUT_PKTS,SERVER_TCP_FLAGS,SRC_TO_DST_SECOND_BYTES,DST_TO_SRC_SECOND_BYTES,RETRANSMITTED_IN_BYTES,...,NUM_PKTS_UP_TO_128_BYTES,NUM_PKTS_128_TO_256_BYTES,NUM_PKTS_256_TO_512_BYTES,NUM_PKTS_512_TO_1024_BYTES,NUM_PKTS_1024_TO_1514_BYTES,TCP_WIN_MAX_OUT,DNS_QUERY_TYPE,DNS_TTL_ANSWER,IPV4_SRC_ADDR_INT,IIPV4_DST_ADDR_INT
6,0,0,318,3,0,0,0,318.0,0.0,0,...,3,0,0,0,0,0,0,0,3232235809,3232235969
11,0,0,318,3,0,0,0,318.0,0.0,0,...,3,0,0,0,0,0,0,0,3232235813,3232235969
17,42007,135,392,6,276,4,27,392.0,276.0,0,...,10,0,0,0,0,8192,0,0,3232235813,3232235969
55,40151,135,392,6,276,4,27,392.0,276.0,0,...,10,0,0,0,0,8192,0,0,3232235809,3232235969
83,36609,135,392,6,276,4,27,392.0,276.0,0,...,10,0,0,0,0,8192,0,0,3232235809,3232235969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16940383,0,0,136,2,0,0,0,136.0,0.0,0,...,2,0,0,0,0,0,0,0,3232235810,3232235969
16940392,0,0,84,1,0,0,0,84.0,0.0,0,...,1,0,0,0,0,0,0,0,3232235807,3232235966
16940440,38331,135,60,1,60,1,18,60.0,60.0,0,...,2,0,0,0,0,8192,0,0,3232235809,3232235969
16940467,35647,445,157,2,197,3,28,157.0,197.0,0,...,5,0,0,0,0,259,0,0,3232235813,3232235969


In [5]:
y

6           0
11          0
17          9
55          9
83          9
           ..
16940383    0
16940392    0
16940440    9
16940467    9
16940477    9
Name: Attack_Num, Length: 13374327, dtype: int64

In [4]:
X.to_csv("X_data.csv", index=False)
y.to_csv("y_data.csv", index=False)

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
model = ExtraTreesClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)
classification_rep = classification_report(y_test, y_test_pred)
conf_matrix = confusion_matrix(y_test, y_test_pred)

print(f"Training Accuracy: {train_accuracy:.4f}")
print(f"Testing Accuracy: {test_accuracy:.4f}")
print("\nClassification Report:\n", classification_rep)
print("\nConfusion Matrix:\n", conf_matrix)

In [ ]:
feature_importance = model.feature_importances_

In [ ]:
nan_counts = data.isna().sum()
print(nan_counts)

In [ ]:
data.info()

In [ ]:
plt.figure(figsize=(15, 8))
data.boxplot(rot=90)  
plt.show()

In [ ]:
def detect_outliers_iqr(df):
    outliers = {}
    for col in df.select_dtypes(include=['int64', 'float64']).columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers[col] = df[(df[col] < lower_bound) | (df[col] > upper_bound)].shape[0]
    
    return outliers

outliers = detect_outliers_iqr(data)
print(outliers)

In [ ]:
data = pd.DataFrame([{
    'L4_SRC_PORT': 1243273, 'L4_DST_PORT': 2896998, 'IN_BYTES': 2277264,
    'IN_PKTS': 1087884, 'OUT_BYTES': 3124472, 'OUT_PKTS': 1070461,
    'SERVER_TCP_FLAGS': 378, 'SRC_TO_DST_SECOND_BYTES': 2176653,
    'DST_TO_SRC_SECOND_BYTES': 3340528, 'RETRANSMITTED_IN_BYTES': 262886,
    'RETRANSMITTED_IN_PKTS': 262886, 'RETRANSMITTED_OUT_BYTES': 313470,
    'RETRANSMITTED_OUT_PKTS': 313470, 'NUM_PKTS_UP_TO_128_BYTES': 664493,
    'NUM_PKTS_128_TO_256_BYTES': 1579644, 'NUM_PKTS_256_TO_512_BYTES': 2357972,
    'NUM_PKTS_512_TO_1024_BYTES': 1780943, 'NUM_PKTS_1024_TO_1514_BYTES': 2064921,
    'TCP_WIN_MAX_OUT': 0, 'DNS_QUERY_TYPE': 2398182, 'DNS_TTL_ANSWER': 1106660,
    'IPV4_SRC_ADDR_INT': 1351532, 'IIPV4_DST_ADDR_INT': 1651379, 'Attack_Num': 11148
}])
data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)  # না হলে data.fillna(method='ffill', inplace=True)

In [ ]:
data = data.astype(np.float64)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

iso_forest = IsolationForest(contamination=0.05, random_state=42)
outliers_iso = iso_forest.fit_predict(scaled_data)

In [ ]:
import pandas as pd

# ফিল্টার করা numpy array থেকে DataFrame তৈরি করা
filtered_data = pd.DataFrame(data[outliers_iso == 1], columns=data.columns)

# ফিচার এবং টার্গেট আলাদা করা
X = filtered_data.drop(columns=['Attack_Num'])
y = filtered_data['Attack_Num']



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# for col in data.select_dtypes(include=['int64']).columns:
#     data[col] = data[col].astype(np.float64)

In [ ]:
# data.replace([np.inf, -np.inf], np.nan, inplace=True)
data.dropna(inplace=True)

In [ ]:
target_column = "Attack_Num"
X = data.drop(columns=[target_column])
y = data[target_column].astype(np.float64)

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
nan_counts = data.isna().sum()
print(nan_counts)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)

In [ ]:
# regressors = {
#     "RandomForest": RandomForestRegressor(n_estimators=500, random_state=42),
#     "ExtraTrees": ExtraTreesRegressor(n_estimators=500, random_state=42),
#     "GradientBoosting": GradientBoostingRegressor(n_estimators=500, random_state=42),
#     "Bagging": BaggingRegressor(random_state=42),
#     "HistGradientBoosting": HistGradientBoostingRegressor(max_iter=1000, learning_rate=0.1, max_leaf_nodes=40, random_state=42),
#     "DecisionTree": DecisionTreeRegressor(random_state=42),
#     "LinearRegression": LinearRegression(),
#     "Ridge": Ridge(alpha=1.0),
#     "Lasso": Lasso(alpha=0.1),
#     "KNeighbors": KNeighborsRegressor(),
#     "SVR": SVR(),
#     "XGBoost": XGBRegressor(n_estimators=500, learning_rate=0.1, random_state=42),
#     "LightGBM": LGBMRegressor(n_estimators=500, learning_rate=0.1, random_state=42)
# }

# # Model evaluation function
# def evaluate_regressors(regressors, x_train, x_test, y_train, y_test):
#     results = []
#     for name, reg in regressors.items():
#         try:
#             reg.fit(x_train, y_train)
#             y_pred = reg.predict(x_test)

#             metrics = {
#                 'Model': name,
#                 'MAE': mean_absolute_error(y_test, y_pred),
#                 'MSE': mean_squared_error(y_test, y_pred),
#                 'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
#                 'R² Score': r2_score(y_test, y_pred)
#             }
#             results.append(metrics)
#         except Exception as e:
#             print(f"Error with regressor {name}: {e}")
#     return pd.DataFrame(results)

# results_df = evaluate_regressors(regressors, x_train, x_test, y_train, y_test)
# print(results_df)

# #results_df.to_csv("RegressionResults_TestSize=0.1.csv", index=False)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import (
    RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, BaggingRegressor
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

df = pd.read_csv('/kaggle/input/iiiiotv2/cleaned_data.csv') 
df = df.astype({col: 'int32' for col in df.select_dtypes(include=['int64'])})
df = df.astype({col: 'float32' for col in df.select_dtypes(include=['float64'])})

def check_for_infs_and_nans(df):
    infs = np.isinf(df).sum().sum()  
    nans = df.isnull().sum().sum() 
    return infs, nans

def drop_infs_and_nans(df):
    df.replace([np.inf, -np.inf], np.nan, inplace=True)  
    df.dropna(inplace=True)  
    return df

infs, nans = check_for_infs_and_nans(df)

if infs > 0 or nans > 0:
    print(f"Dataset contains {infs} infinite values and {nans} NaN values. Dropping affected rows...")
    df = drop_infs_and_nans(df)

X = df.drop('Attack_Num', axis=1)  
y = df['Attack_Num'] 

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

regressors = {
    "RandomForest": RandomForestRegressor(n_estimators=500, random_state=42),
    "ExtraTrees": ExtraTreesRegressor(n_estimators=500, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=500, random_state=42),
    "Bagging": BaggingRegressor(random_state=42),
    "DecisionTree": DecisionTreeRegressor(random_state=42),
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(alpha=1.0),
    "Lasso": Lasso(alpha=0.1),
    "KNeighbors": KNeighborsRegressor(),
    "SVR": SVR(),
    "XGBoost": XGBRegressor(n_estimators=500, learning_rate=0.1, random_state=42),
    "LightGBM": LGBMRegressor(n_estimators=500, learning_rate=0.1, random_state=42)
}

# Model evaluation function
def evaluate_regressors(regressors, x_train, x_test, y_train, y_test):
    results = []
    for name, reg in regressors.items():
        try:
            reg.fit(x_train, y_train)
            y_pred = reg.predict(x_test)

            metrics = {
                'Model': name,
                'MAE': mean_absolute_error(y_test, y_pred),
                'MSE': mean_squared_error(y_test, y_pred),
                'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
                'R² Score': r2_score(y_test, y_pred)
            }
            results.append(metrics)
        except Exception as e:
            print(f"Error with regressor {name}: {e}")
    return pd.DataFrame(results)

results_df = evaluate_regressors(regressors, x_train, x_test, y_train, y_test)
print(results_df)

results_df.to_csv("RegressionResults_TestSize=0.1.csv", index=False)